# 지하철 위치정보 + 지하철 시간대별 승하차
- 강남구 내 지하철만 추출

# 1차 시도(연습)
- 일부 역에 2개인 위치정보 확인 >> 데이터 재 수집

In [43]:
import warnings
warnings.filterwarnings('ignore')

In [37]:
import requests
import pandas as pd


base_url = "http://openapi.seoul.go.kr:8088/6253514753727330363652684a7172/json/subwayStationMaster/"

items_per_page = 1000
total_pages = None
data_frames = []

for i in range(1, 1001):
    start_page = (i - 1) * items_per_page + 1
    end_page = i * items_per_page
    url = f"{base_url}{start_page}/{end_page}/"
    
    response = requests.get(url)
    data = response.json()

    items = data.get('subwayStationMaster', {}).get('row', [])
    if not items:
        break
    
    df = pd.DataFrame(items)
    data_frames.append(df)

# Concatenate all DataFrames into a single DataFrame
combined_df = pd.concat(data_frames, ignore_index=True)

# Now you have a single DataFrame containing data from all pages
combined_df.head()

,STATN_ID,STATN_NM,ROUTE,CRDNT_Y,CRDNT_X
0,0150,서울역,1호선,37.556228,126.972135
1,0151,시청,1호선,37.565715,126.977088
2,0152,종각,1호선,37.570161,126.982923
3,0153,종로3가,1호선,37.570406,126.991847
4,0154,종로5가,1호선,37.570926,127.001849


In [38]:
combined_df.columns = ['역사_ID', '지하철역', '호선', '위도', '경도']
combined_df.head()

,역사_ID,지하철역,호선,위도,경도
0,0150,서울역,1호선,37.556228,126.972135
1,0151,시청,1호선,37.565715,126.977088
2,0152,종각,1호선,37.570161,126.982923
3,0153,종로3가,1호선,37.570406,126.991847
4,0154,종로5가,1호선,37.570926,127.001849


In [12]:
df2 = pd.read_csv('강남구_분기별_시간대별_지하철승하차수.csv')
df2.head()

,분기,사용월,호선명,지하철역,00-06시 승하차,06-11시 승하차,11-14시 승하차,14-17시 승하차,17-21시 승하차,21-24시 승하차
0,1,202201,2호선,삼성(무역센터),9526.0,812881.0,311547.0,396843.0,838839.0,176400.0
1,1,202201,2호선,선릉,8897.0,780576.0,305703.0,392517.0,782659.0,202932.0
2,1,202201,2호선,역삼,9511.0,935225.0,283978.0,349351.0,815525.0,189429.0
3,1,202201,2호선,강남,15131.0,1128475.0,611200.0,757998.0,1353366.0,475275.0
4,1,202201,3호선,압구정,4856.0,493226.0,264527.0,306262.0,532479.0,123276.0


In [14]:
result = pd.merge(df2, combined_df, how='left', on='지하철역')
result.head()

,분기,사용월,호선명,지하철역,00-06시 승하차,06-11시 승하차,11-14시 승하차,14-17시 승하차,17-21시 승하차,21-24시 승하차,역사_ID,호선,위도,경도
0,1,202201,2호선,삼성(무역센터),9526.0,812881.0,311547.0,396843.0,838839.0,176400.0,0219,2호선,37.508844,127.06316
1,1,202201,2호선,선릉,8897.0,780576.0,305703.0,392517.0,782659.0,202932.0,0220,2호선,37.504286,127.048203
2,1,202201,2호선,선릉,8897.0,780576.0,305703.0,392517.0,782659.0,202932.0,1023,분당선,37.504856,127.048807
3,1,202201,2호선,역삼,9511.0,935225.0,283978.0,349351.0,815525.0,189429.0,0221,2호선,37.500622,127.036456
4,1,202201,2호선,강남,15131.0,1128475.0,611200.0,757998.0,1353366.0,475275.0,0222,2호선,37.49799,127.027912


In [15]:
# 확인용
combined_df[combined_df['지하철역'] == '선릉']

,역사_ID,지하철역,호선,위도,경도
29,0220,선릉,2호선,37.504286,127.048203
145,1023,선릉,분당선,37.504856,127.048807


In [16]:
# 확인용
combined_df[combined_df['지하철역'] == '삼성(무역센터)']

,역사_ID,지하철역,호선,위도,경도
28,0219,삼성(무역센터),2호선,37.508844,127.06316


## 강남구 지하철 위경도 데이터만 분리

In [51]:
combined_df.head()

,역사_ID,지하철역,호선,위도,경도
0,0150,서울역,1호선,37.556228,126.972135
1,0151,시청,1호선,37.565715,126.977088
2,0152,종각,1호선,37.570161,126.982923
3,0153,종로3가,1호선,37.570406,126.991847
4,0154,종로5가,1호선,37.570926,127.001849


In [52]:
sub_gangnam = pd.read_csv('gangnam_sub_list.csv')
sub_gangnam.head()

,역명,호선
0,삼성(무역센터),2
1,선릉,2
2,역삼,2
3,강남,2
4,압구정,3


In [56]:
sub_gangnam.rename(columns = {'역명':'지하철역'}, inplace = True)

result_2 = pd.merge(sub_gangnam, combined_df, how='left', on='지하철역')
result_2.head()

,지하철역,호선_x,역사_ID,호선_y,위도,경도
0,삼성(무역센터),2,0219,2호선,37.508844,127.06316
1,선릉,2,0220,2호선,37.504286,127.048203
2,선릉,2,1023,분당선,37.504856,127.048807
3,역삼,2,0221,2호선,37.500622,127.036456
4,강남,2,0222,2호선,37.49799,127.027912


In [60]:
result_cols = ['지하철역','호선_x', '위도', '경도']
result_2 = result_2[result_cols]
result_2.rename(columns={'호선_x':'호선'}, inplace = True)

result_2.head()

,지하철역,호선,위도,경도
0,삼성(무역센터),2,37.508844,127.06316
1,선릉,2,37.504286,127.048203
2,선릉,2,37.504856,127.048807
3,역삼,2,37.500622,127.036456
4,강남,2,37.49799,127.027912


In [61]:
result_2.to_csv('강남구_지하철역_위경도.csv', index=False)

# 2차 시도(최종) 교통데이터 빅데이터에서 지하철 위치 크롤링

- api : 7ee56846-6e0d-4592-a7ec-65c34e69afd1
- https://t-data.seoul.go.kr/category/dataviewopenapi.do?data_id=1036

In [1]:
import requests

# API 키
apikey = '7ee56846-6e0d-4592-a7ec-65c34e69afd1'

# API 엔드포인트와 쿼리 매개변수 설정
base_url = 'https://t-data.seoul.go.kr/apig/apiman-gateway/tapi/TaimsKsccDvSubwayStationGeom/1.0'
params = {
    'apiKey': apikey,
    'startRow': '1',
    'rowCnt': '100000'
}

# GET 요청 보내기
req = requests.get(base_url, params=params)

# 응답 JSON 파싱
json_df = req.json()

"""[{'outStnNum': '4128',
  'stnKrNm': '삼성중앙',
  'lineNm': '9호선(연장)',
  'convX': '127.053282',
  'convY': '37.513011'},
 {'outStnNum': '4124',
  'stnKrNm': '사평',
  'lineNm': '9호선',
  'convX': '127.015259',
  'convY': '37.504206'},"""
    
import pandas as pd
sub = pd.DataFrame(json_df)
sub

,outStnNum,stnKrNm,lineNm,convX,convY
0,4128,삼성중앙,9호선(연장),127.053282,37.513011
1,4124,사평,9호선,127.015259,37.504206
2,4121,구반포,9호선,126.987332,37.501364
3,4119,흑석(중앙대입구),9호선,126.963708,37.50877
4,4116,샛강,9호선,126.928422,37.517274
...,...,...,...,...,...
760,4411,관악산(서울대),신림선,126.9450639,37.4691018
761,4306,신논현,신분당선(연장2),127.02506,37.504598
762,1404,탕정,장항선,127.08485,36.78866
763,4304,신사,신분당선(연장2),127.020114,37.516334


In [2]:
sub.drop('outStnNum', axis = 1, inplace = True)
sub.rename(columns = {'stnKrNm':'지하철역', 'convX':'위도', 'convY':'경도', 'lineNm':'호선명'}, inplace = True)
sub.head()

,지하철역,호선명,위도,경도
0,삼성중앙,9호선(연장),127.053282,37.513011
1,사평,9호선,127.015259,37.504206
2,구반포,9호선,126.987332,37.501364
3,흑석(중앙대입구),9호선,126.963708,37.50877
4,샛강,9호선,126.928422,37.517274


### (1)강남구 역만 추출

In [3]:
station_list = [
    "강남구청",
    "강남",
    "개포동",
    "구룡",
    "논현",
    "대모산입구",
    "대청",
    "대치",
    "도곡",
    "매봉",
    "봉은사",
    "삼성(무역센터)",
    "삼성중앙",
    "선릉",
    "선정릉",
    "수서",
    "신논현",
    "신사",
    "압구정로데오",
    "압구정",
    "양재",
    "언주",
    "역삼",
    "일원",
    "청담",
    "학동",
    "학여울",
    "한티"
]

gangnam_sub = sub[sub['지하철역'].isin(station_list)]
gangnam_sub.head()

,지하철역,호선명,위도,경도
0,삼성중앙,9호선(연장),127.053282,37.513011
30,강남구청,분당선,127.041151,37.517469
43,압구정,3호선,127.028461,37.527072
53,강남,신분당선,127.028104,37.496837
111,봉은사,9호선(연장),127.060245,37.514219


### (2)위도/경도 세트가 2개 이상인 것만 추출
- 전체 28개 역 중, 9개 역

In [4]:
len(station_list)

28

In [5]:
station_count = gangnam_sub['지하철역'].value_counts()

station_2pos = station_count[station_count >= 2].index.tolist()
station_2ps_df = gangnam_sub[gangnam_sub['지하철역'].isin(station_2pos)]
station_2ps_df.sort_values('지하철역')

,지하철역,호선명,위도,경도
53,강남,신분당선,127.028104,37.496837
351,강남,2호선,127.027912,37.49799
30,강남구청,분당선,127.041151,37.517469
544,강남구청,7호선,127.041255,37.517179
764,논현,신분당선(연장2),127.021415,37.511093
317,논현,7호선,127.021415,37.511093
632,도곡,분당선,127.055186,37.491224
683,도곡,3호선,127.055452,37.490922
396,선릉,분당선,127.048807,37.504856
613,선릉,2호선,127.048203,37.504286


In [6]:
station2s = station_2ps_df.drop('호선명', axis = 1)
station2s.reset_index(drop=True)

,지하철역,위도,경도
0,강남구청,127.041151,37.517469
1,강남,127.028104,37.496837
2,선정릉,127.043999,37.510297
3,신논현,127.02506,37.504598
4,논현,127.021415,37.511093
5,강남,127.027912,37.49799
6,수서,127.101422,37.487472
7,선릉,127.048807,37.504856
8,선정릉,127.043677,37.510735
9,신사,127.020114,37.516334


In [7]:
station2s[['위도', '경도']] = station2s[['위도', '경도']].astype(float)

In [8]:
double_positions = station2s.groupby('지하철역')[['위도', '경도']].mean()
display(len(double_positions), double_positions)

9

,위도,경도
지하철역,,
강남,127.028008,37.497414
강남구청,127.041203,37.517324
논현,127.021415,37.511093
도곡,127.055319,37.491073
선릉,127.048505,37.504571
선정릉,127.043838,37.510516
수서,127.101664,37.487425
신논현,127.025060,37.504598
신사,127.020114,37.516334


In [9]:
#선릉으로 테스트
(127.048807 + 127.048203)/2

127.048505

In [10]:
double_positions = double_positions.reset_index()
double_positions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9 entries, 0 to 8
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지하철역    9 non-null      object 
 1   위도      9 non-null      float64
 2   경도      9 non-null      float64
dtypes: float64(2), object(1)
memory usage: 344.0+ bytes


### (3)위도/경도가 하나의 값인 강남 지하철역

In [11]:
double_stations = double_positions['지하철역'].values.tolist()
double_stations

['강남', '강남구청', '논현', '도곡', '선릉', '선정릉', '수서', '신논현', '신사']

In [12]:
station_list = [
    "강남구청",
    "강남",
    "개포동",
    "구룡",
    "논현",
    "대모산입구",
    "대청",
    "대치",
    "도곡",
    "매봉",
    "봉은사",
    "삼성(무역센터)",
    "삼성중앙",
    "선릉",
    "선정릉",
    "수서",
    "신논현",
    "신사",
    "압구정로데오",
    "압구정",
    "양재",
    "언주",
    "역삼",
    "일원",
    "청담",
    "학동",
    "학여울",
    "한티"
]
# len(station_list)

single_stations = list(set(station_list) - set(double_stations))
single_stations

len(single_stations)

19

In [14]:
single_positions = gangnam_sub[gangnam_sub['지하철역'].isin(single_stations)]
# single_positions['지하철역'].value_counts()
single_positions.drop('호선명', axis = 1, inplace = True)
display(single_positions.head(), len(single_positions))

/var/folders/hj/1c4sv0bn28v5lr5g0s_fr4w80000gn/T/ipykernel_22394/1407137177.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_positions.drop('호선명', axis = 1, inplace = True)


,지하철역,위도,경도
0,삼성중앙,127.053282,37.513011
43,압구정,127.028461,37.527072
111,봉은사,127.060245,37.514219
113,언주,127.033868,37.507287
178,압구정로데오,127.040534,37.527381


18

In [16]:
single_positions[['위도', '경도']] = single_positions[['위도', '경도']].astype(float)
single_positions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 0 to 633
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지하철역    18 non-null     object 
 1   위도      18 non-null     float64
 2   경도      18 non-null     float64
dtypes: float64(2), object(1)
memory usage: 576.0+ bytes


/var/folders/hj/1c4sv0bn28v5lr5g0s_fr4w80000gn/T/ipykernel_22394/1639560964.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  single_positions[['위도', '경도']] = single_positions[['위도', '경도']].astype(float)


In [17]:
#single_positions & double_positions 붙이기

gangnam_sub_positions = pd.concat([single_positions, double_positions], axis = 0, ignore_index = True)
gangnam_sub_positions

,지하철역,위도,경도
0,삼성중앙,127.053282,37.513011
1,압구정,127.028461,37.527072
2,봉은사,127.060245,37.514219
3,언주,127.033868,37.507287
4,압구정로데오,127.040534,37.527381
5,학동,127.031656,37.514229
6,삼성(무역센터),127.063160,37.508844
7,한티,127.052873,37.496237
8,구룡,127.058856,37.486839
9,개포동,127.066140,37.489116


In [18]:
gangnam_sub_positions.to_csv('강남지하철_위도경도_최종.csv', index = False)

In [19]:
gangnam_sub_positions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   지하철역    27 non-null     object 
 1   위도      27 non-null     float64
 2   경도      27 non-null     float64
dtypes: float64(2), object(1)
memory usage: 776.0+ bytes


# 연도분기 지하철승하차 인원 + 지하철 위치 결합

In [20]:
import pandas as pd

data = pd.read_csv('./연도_분기_지하철승하차.csv')
data

,연도,분기,월,지하철역,호선명,00-06시,06-11시,11-14시,14-17시,17-21시,21-24시
0,2020,1,1,강남,2호선,64195,1390569,851787,1020723,1842601,807774
1,2020,1,1,삼성(무역센터),2호선,20837,968174,439256,553549,1077187,279793
2,2020,1,1,선릉,2호선,23811,891141,372616,472332,898156,267683
3,2020,1,1,역삼,2호선,20816,1001912,343193,410819,900137,237617
4,2020,1,1,대청,3호선,8088,163454,74159,87450,159204,55764
...,...,...,...,...,...,...,...,...,...,...,...
1147,2022,4,12,선릉,분당선,3548,359856,108569,120650,261408,66169
1148,2022,4,12,선정릉,분당선,2736,149375,54523,63212,133686,29270
1149,2022,4,12,수서,분당선,8867,255894,124871,145026,250427,80251
1150,2022,4,12,압구정로데오,분당선,5681,310303,180327,232018,449446,127815


In [21]:
merged_df = pd.merge(data, gangnam_sub_positions, how='left', on='지하철역')
merged_df

,연도,분기,월,지하철역,호선명,00-06시,06-11시,11-14시,14-17시,17-21시,21-24시,위도,경도
0,2020,1,1,강남,2호선,64195,1390569,851787,1020723,1842601,807774,127.028008,37.497414
1,2020,1,1,삼성(무역센터),2호선,20837,968174,439256,553549,1077187,279793,127.063160,37.508844
2,2020,1,1,선릉,2호선,23811,891141,372616,472332,898156,267683,127.048505,37.504571
3,2020,1,1,역삼,2호선,20816,1001912,343193,410819,900137,237617,127.036456,37.500622
4,2020,1,1,대청,3호선,8088,163454,74159,87450,159204,55764,127.079532,37.493514
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2022,4,12,선릉,분당선,3548,359856,108569,120650,261408,66169,127.048505,37.504571
1148,2022,4,12,선정릉,분당선,2736,149375,54523,63212,133686,29270,127.043838,37.510516
1149,2022,4,12,수서,분당선,8867,255894,124871,145026,250427,80251,127.101664,37.487425
1150,2022,4,12,압구정로데오,분당선,5681,310303,180327,232018,449446,127815,127.040534,37.527381


In [22]:
merged_df[merged_df['지하철역']=='선릉']

,연도,분기,월,지하철역,호선명,00-06시,06-11시,11-14시,14-17시,17-21시,21-24시,위도,경도
2,2020,1,1,선릉,2호선,23811,891141,372616,472332,898156,267683,127.048505,37.504571
27,2020,1,1,선릉,분당선,3798,342287,105938,112244,247950,64106,127.048505,37.504571
34,2020,1,2,선릉,2호선,20643,838792,313685,403230,833708,221182,127.048505,37.504571
59,2020,1,2,선릉,분당선,3303,322885,89308,95273,229136,52034,127.048505,37.504571
66,2020,1,3,선릉,2호선,18316,749430,243748,334871,721523,178110,127.048505,37.504571
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1083,2022,4,10,선릉,분당선,3625,339408,106766,112229,241001,56090,127.048505,37.504571
1090,2022,4,11,선릉,2호선,20975,945357,364616,451201,919842,232863,127.048505,37.504571
1115,2022,4,11,선릉,분당선,3633,370996,106178,116249,266240,56982,127.048505,37.504571
1122,2022,4,12,선릉,2호선,22337,914313,372274,466563,908477,267347,127.048505,37.504571


In [23]:
merged_df.to_csv('최종_지하철_3개년_승하차.csv', index = False)

In [24]:
position_merge = merged_df.copy()

In [25]:
position_merge['위경도'] = '(' + position_merge['경도'].astype(str) + ',' + position_merge['위도'].astype(str) + ')'

In [26]:
position_merge

,연도,분기,월,지하철역,호선명,00-06시,06-11시,11-14시,14-17시,17-21시,21-24시,위도,경도,위경도
0,2020,1,1,강남,2호선,64195,1390569,851787,1020723,1842601,807774,127.028008,37.497414,"(37.4974135,127.028008)"
1,2020,1,1,삼성(무역센터),2호선,20837,968174,439256,553549,1077187,279793,127.063160,37.508844,"(37.508844,127.06316)"
2,2020,1,1,선릉,2호선,23811,891141,372616,472332,898156,267683,127.048505,37.504571,"(37.504571,127.048505)"
3,2020,1,1,역삼,2호선,20816,1001912,343193,410819,900137,237617,127.036456,37.500622,"(37.500622,127.036456)"
4,2020,1,1,대청,3호선,8088,163454,74159,87450,159204,55764,127.079532,37.493514,"(37.493514,127.079532)"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1147,2022,4,12,선릉,분당선,3548,359856,108569,120650,261408,66169,127.048505,37.504571,"(37.504571,127.048505)"
1148,2022,4,12,선정릉,분당선,2736,149375,54523,63212,133686,29270,127.043838,37.510516,"(37.510515999999996,127.043838)"
1149,2022,4,12,수서,분당선,8867,255894,124871,145026,250427,80251,127.101664,37.487425,"(37.487425,127.1016645)"
1150,2022,4,12,압구정로데오,분당선,5681,310303,180327,232018,449446,127815,127.040534,37.527381,"(37.527381,127.040534)"
